# Ayudantía Threading
**Autores: Pablo Olea y Enzo Tamburini**

## Restaurante hackeado 

### Introducción

¡Oh no!, empezaron a llegar los comensales al restaurante PrograChef y la receta secreta aún está encriptada (debido a que todos los cocineros son programadores. Los únicos que pueden ayudarnos son los Chef Genios, quienes son muy torpes abriendo el sobre pero muy buenos resolviendo el enigma. 

Para esto, habrán tres equipos quienes estarán conformados por un mozo, un cocinero y un genio. El primero entregará la orden de platos (serán hartos) a hacer previo al gran plato principal, cuya receta está encriptada y que el Chef Genio tendrá que descifrar. Una vez que ambos terminen, el mozo entregará el plato y la simulación terminará. El primer equipo que entregue el plato será el ganador :D 


### Entidades

A los cocineros les llegan órdenes distintas con cantidades de platos que varían entre 60 a 80 platos antes de que puedan hacer el plato principal, que será decodificado por el **Chef Genio**. 

- **Chef Genio**: Este personaje será el encargado de descifrar la receta que dejaron codificada con clave _agujeritos_. El genio tendrá un nombre y lo gritará al terminar su solución. Como el sobre en el que viene el
problema está tan bien sellado, el genio tardará entre 5 a 20 _unidades temporales_ en abrirlo, para recién comenzar
a resolverlo. Una vez que el genio abre el sobre, puede resolver el problema de forma instantánea (porque es un genio! :D). Al terminar, el genio avisa al **Mozo** que ya está listo.

- **Cocinero**: Este personaje se encargará de hacer los otros platos que están en esta y otras órdenes. Los hará a una velocidad de 3 - 8 platos por _unidad temporal_. Al cocinero por un tema de cansancio se le puede caer el plato con probabilidad **0.08** en **cualquier momento**. Cuando esto sucede, el **Cocinero** deberá llamar al **Barrendero** para que limpie la estación de trabajo. Al terminar, el cocinero avisa al **Mozo** que ya está listo.

- **Barrendero**: Es el encargado de limpiar la zona de trabajo. Cuando a un cocinero se le caiga un plato, podrá limpiarlo todo en un tiempo de 2 a 5 _unidades temporales_, tiempo en el cual el cocinero deberá esperar a que termine para seguir avanzando. Si a otro chef se le cae el plato, este deberá esperar a que el barrendero se encuentre disponible. Como hay que ir indicando qué sucede en todo momento, por lo que debe indicar el momento en que empieza a limpiar y cuando termina, además decir qué cocinero es el que retoma el trabajo.

- **Mozo**: Este personaje esperará el pedido hecho por su equipo conformado por un cocinero y un chef genio. Una vez que **ambos** terminen, el mozo entregará el plato principal a quien lo pidió, lo cual le tomará entre 3 a 5 _unidades temporales_. Cuando termine de entregar el pedido el mozo tendrá que avisar.


### El Problema

Se nos dijo que quien encriptó la receta secreta era un amante de las claves sencillas y que por lo tanto, probáramos con el código ``Agujeritos``: 

| A | G | U | J | E | R | I | T | O | S |
|---|---|---|---|---|---|---|---|---|---|
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |

- La clave **agujeritos** consiste en cambiar los números de una palabra por las letras según lo indicado en la tabla. Por ejemplo, la palabra **cuchufli** se transformaría en **c2ch2fl6**.

Una vez que el genio encuentre la solución al problema, este **avisará** que lo logró y escribirá la solución a
un archivo de texto de nombre ``solucion.txt``.

### Archivo

Se entrega un único archivo ``Problema.txt`` con la receta que deben desencriptar los genios.

### Registro

Una vez finalizada la simulación, deberás imprimir las estadísticas, en donde cada grupo debe indicar:

- Nombre del cocinero
- Cantidad de platos cocinados
- Nombrar al chef genio, quien se presentará y dirá si resolvió el problema o no.
- Finalmente, indicar el tiempo que se demoró en terminar el **primer** grupo.

### Librerías y constantes
Primero llamamos a las librerías que usaremos en el programa y ademas declaramos las constantes que usaremos para el resto del programa

In [1]:
import threading 
import random 
import time

DEMORA_SOBRE = (5, 20)
CANTIDAD_ORDENES = (60, 80)
VELOCIDAD_TRABAJO = (3, 8)
TIEMPO_BARRENDERO = (2, 5)
DEMORA_MOZO = (3, 5)
PROBABILIDAD_ROMPER_PLATOS = 0.08

Se debe entender que en esta simulación, cada una de las entidades debe actuar de manera independiente, sólo interactuando entre ellas según dice el enunciado y por ende cada una de estas sería un ``Thread`` al cual hay que personalizar. También acotar de que en cada caso, se necesita que estos ``Thread`` **deben** ser ``Daemon`` debido a que una vez que uno de los equipos logra terminar, inmediatamente deja de importarnos el resto y el sistema termina todos los ``Thread`` que aún no terminan.

## ChefGenio

> - Tiene nombre

Esta entidad sólo debe guardar un **nombre** por el enunciado. Siendo más precisos, el genio se demora en abrir el sobre, por lo que **cada instancia debe tener su propio valor** aleatorio. 

> - Solo un genio a la vez puede estar usando el archivo ``Problema.txt`` 

Esto indica que todos los genios accederán al mismo archivo. Por lo que presenta una _zona crítica_ y que implica el uso de un `Lock()` compartido por la clase (es decir, uno solo por clase).
    
> - Solo un genio a la vez puede escribir el archivo receta.txt 

Misma acotación. Lo que se puede decir, es que si los genios demorasen menos en leer el archivo y se ocupara el mismo `Lock()` anterior, podría tener problemas al tener que esperar que termine este proceso. Lo anterior se resume en que es mejor ocupar un `Thread` por zona crítica.

> - Le avisa al **Mozo** cuando está listo.

Con esto, debe existir una forma de que el `Mozo` espere al genio. Una de las posibles soluciones es utilizar un `Event()` que sea compartido entre esta entidad y la del `Mozo` cosa que este último espere a que el flag cambie de valor y así pueda avanzar. Esto "simularía" el aviso que debe darle el Genio al ``Mozo``. Como es algo compartido, sería mejor darle el evento antes de crearlo dentro de la instancia (de otra forma, ¿cómo se la darías al `Mozo`?).

In [2]:
class ChefGenio(threading.Thread):

    problema_lock = threading.Lock()
    solucion_lock = threading.Lock()

    def __init__(self, nombre, evento_genio):
        super().__init__()
        self.nombre = nombre
        self.demora_sobre = random.randint(*DEMORA_SOBRE)
        self.receta_descifrada = False
        self.daemon = True
        self.evento_genio = evento_genio

    @staticmethod
    def descifrar_receta():
        # Este método es sólo para descifrar la receta y se puede modelar como uno mejor considere conveniente
        clave = "agujeritos"
        numeros = "0123456789"
        solucion = ""
        with open("Problema.txt", mode="r", encoding="UTF-8") as file:
            for line in file:
                for s in line:
                    if s.lower() in numeros:
                        solucion += clave[int(s)]
                    elif s.lower() in clave:
                        letra = clave.index(s)
                        solucion += letra
                    else:
                        solucion += s
        return solucion

    def run(self):
        time.sleep(self.demora_sobre)

        # controlamos que sólo un genio a la vez use el archivo problema.txt
        with self.problema_lock:
            receta = self.descifrar_receta()

        # controlamos que solo un genio a la vez escriba el archivo receta.txt
        with self.solucion_lock:
            # para propósitos del modelo, no importa que los genios sobreescriban el archio receta.txt si ya existe
            with open("receta.txt", mode="w", encoding="UTF-8") as file:
                file.write(receta)
        self.receta_descifrada = True
        print("{} descifró la receta".format(self.nombre))
        self.evento_genio.set() # aqui avisa que terminó su trabajo
                

## Cocinero
> - Tiene nombre
> - Le avisa al **Mozo** cuando está listo.

Al igual que en la entidad anterior, nuestra clase `Cocinero` tendrá inputs parecidos, recibirá un nombre y a la vez un `Event()` que será compartido con la clase `Mozo` para ser avisado de forma similar a la anteriormente descrita en la clase anterior. **Aclaración:** Se reciben dos instancias de eventos clase `Event` **distintas**, si fueran iguales tendrías el mismo problema.

> - Tiene una cantidad de ordenes que realizar
> - Tiene una tasa de velocidad de trabajo (platos por unidad temporal)

Estos tres puntos pueden verse como atributos de una instancia de `Cocinero` debido a que varían entre instancias. En cada uno de los atributos se aplica la misma técnica; ``random.randint(*CONSTANTE_APROPIADA)`` dado que en todos los casos es aleatorio para las instancias. 

> - Sabe cuántos platos lleva hechos

Esto se podría hacer de varias maneras, aquí asignamos un atributo que vaya sumando, pero nada impide a que se haga de otra manera (como restando en la magnitud de la orden).

> - Se le pueden caer los platos y llamar al barrendero, el barrendero solo puede ayudar a un cocinero a la vez

Finalmente, el `Barrendero` podría verse de muchas maneras, algunos propusieron un `Thread` pero la verdad aquí bastaba un `Lock` compartido por toda la clase dado que sólo existía un ``Barrendero``. 

In [3]:
class Cocinero(threading.Thread):
    
    # El barrendero es solo un lock. No tiene por que ser un thread
    barrendero_lock = threading.Lock()

    def __init__(self, nombre, evento_cocinero):
        super().__init__()
        self.daemon = True
        self.nombre = nombre
        self.platos_hechos = 0
        self.ordenes = random.randint(*CANTIDAD_ORDENES)
        self.velocidad_trabajo = random.randint(*VELOCIDAD_TRABAJO)
        self.evento_cocinero = evento_cocinero

    def llamar_barrendero(self):
        # llamamos al lock del barrendero, de manera que si a otro cocinero se le rompen los platos, se quede esperando
        # a que se libere el lock
        with self.barrendero_lock:
            print("El barrendero esta limpiando la zona de trabajo de {}".format(self.nombre))
            time.sleep(random.randint(*TIEMPO_BARRENDERO))
            print("El barrendero terminó, {} vuelve al trabajo".format(self.nombre))

    def run(self):
        while True:
            # Notemos que la simulacion va por unidad temporal, no por unidad de plato.  
            time.sleep(1)
            self.platos_hechos += self.velocidad_trabajo # agregamos la cantidad de platos determinados por su tasa de trabajo
            if self.platos_hechos >= self.ordenes: 
                # si llegó a la cantidad de ordenes que debía terminar, entonces terminó su trabajo
                print("{} terminó su trabajo".format(self.nombre))
                break
            if random.random() <= PROBABILIDAD_ROMPER_PLATOS:
                print("Al cocinero {} se le cayeron los platos".format(self.nombre))
                self.llamar_barrendero()
            
        self.evento_cocinero.set() # avisa que terminó su trabajo

## Mozo
>- Tiene nombre

Esta entidad debe guardar un nombre por el enunciado. Esto es para que podamos diferenciar a los mozos activos.

>- Entregará el plato principal a quien lo pidió, lo cual le tomará entre 3 a 5 unidades temporales

Esta sección nos indica que cada mozo tiene un tiempo aleatorio de demora (DEMORA_MOZO) para hacer entrega del plato principal, lo que podemos simular con un *time.sleep()* una vez que el cocinero y el genio avisen que terminaron.

>- Solo entrega el plato principal cuando el **cocinero** y **genio** de su equipo terminaron.

El mozo realizará la entrega una vez que los miembros de su equipo hayan terminado sus trabajos, dicho de otra forma, el mozo deberá **esperar** a recibir un aviso de los threads **cocinero** y **genio** (del mismo equipo). Congruente a lo mencionado en **ChefGenio** y **Cocinero**, sincronizaremos al mozo con las otras entidades por medio de un `Event()`. Tiene que ser un evento_cocinero para sincronizar al mozo con el cocinero(tenemos que pasar este evento a la entidad **mozo** y **cocinero**) y otro evento_genio para sincronizar al mozo con el genio(análogamente, se les pasa este evento a la entidad **mozo** y **genio**).

In [4]:
class Mozo(threading.Thread):

    def __init__(self, nombre, evento_cocinero, evento_genio):
        super().__init__()
        self.nombre = nombre
        self.daemon = True
        self.evento_cocinero = evento_cocinero # mismo evento que se le entrega al cocinero de su equipo
        self.evento_genio = evento_genio # mismo evento que se le entrega al genio de su equipo

    def run(self):
        print("El mozo {} espera al cocinero y al genio".format(self.nombre))
        self.evento_genio.wait() # espera a que el genio termine su trabajo
        self.evento_cocinero.wait() # espera a que el cocinero termine su trabajo
        print("El mozo {} va a realizar la entrega".format(self.nombre))
        time.sleep(random.randint(*DEMORA_MOZO))
        print("El mozo {} termino!!".format(self.nombre))

## Restaurante
>- Solo es para modelacion, no es un thread.

Podemos modelarlo como mejor estimemos conveniente, siempre que se modele de forma correcta la competencia. No es necesario que sea un thread, pero perfectamente se podría hacer de esa forma, considerando que si se desea imprimir resultados en el programa principal, se debe esperar a que termine la simulación, por lo que se tendría que hacer uso de `join()`.

>- Agrega a los equipos que van a competir.

Soló habría que entregarle los nombres de los miembros del equipo que se va a agregar. Notemos que al crear al mozo, cocinero y genio de un mismo equipo, tenemos que crear los eventos que les entregaremos a las entidades para que se puedan sincronizar correctamente.

>- Revisa cuando un equipo hizo la entrega, tiene las estadisticas y anuncia al equipo ganador.

EL formato de impresión de los resultados de la competencia queda a decisión de como uno lo estime mas conveniente, siempre que se impriman todas las estadísticas pedidas.

In [5]:
class Restaurante:
    def __init__(self):
        self.equipos = []
        self.equipo_ganador = None
        self.competencia_terminada = False
        self.tiempo_ganadores = 0

    def agregar_equipo(self, nombre_cocinero, nombre_genio, nombre_mozo):
        # Aquí se crean los eventos que se entregarán a los threads del mismo equipo
        evento_cocinero = threading.Event()
        evento_genio = threading.Event()
        cocinero = Cocinero(nombre_cocinero, evento_cocinero)
        genio = ChefGenio(nombre_genio, evento_genio)
        mozo = Mozo(nombre_mozo, evento_cocinero, evento_genio)
        self.equipos.append((mozo, cocinero, genio)) # agregamos el equipo

    @staticmethod
    def imprimir_info_equipo(equipo):
        mozo, cocinero, genio = equipo
        print("Cocinero: {}".format(cocinero.nombre))
        print("Platos hechos: {}".format(cocinero.platos_hechos))
        if genio.receta_descifrada:
            s = "resolví el problema pues soy una maquina"
        else:
            s = "soy un pecho frío pues no resolví el problema"
        print("Genio: Mi nombre es {} y {}".format(genio.nombre, s))

    
    # Este método es sólo para impresión.
    def resultados(self):
        mozo, cocinero, genio = self.equipo_ganador
        print("---------------------")
        print("Fin de la competencia")
        print("---------------------")
        print("EQUIPO GANADOR: MOZO {} COCINERO {} GENIO {}".format(mozo.nombre, cocinero.nombre, genio.nombre))
        print("TIEMPO DEL EQUIPO GANADOR: {}".format(self.tiempo_ganadores))
        for equipo in self.equipos:
            self.imprimir_info_equipo(equipo)


    def realizar_competencia(self):
        print("-----------------------")
        print("Comienza la competencia")
        print("-----------------------")
        inicio = time.time()
        # damos inicio a los threads de cada equipo
        for mozo, cocinero, genio in self.equipos:
            mozo.start()
            cocinero.start()
            genio.start()
        
        while not self.competencia_terminada:
            for equipo in self.equipos:
                mozo = equipo[0]
                # si un mozo terminó su trabajo, entonces su equipo es el ganador y se detiene la simulación.
                if not mozo.isAlive():
                    self.equipo_ganador = equipo
                    self.competencia_terminada = True
                    self.tiempo_ganadores = time.time() - inicio
                    break

## Sobre la prueba de la simulación
Se recomienda **fuertemente** hacer prueba de este codigo en un editor que no sea jupyter, debido a que jupyter internamente no da por finalizado el programa, por ende los daemon threads no se detienen y siguen vivos despues de terminada la simulación. Si desean realizar la prueba, pueden utilizar la celda de abajo.

In [ ]:
competencia = Restaurante()
competencia.agregar_equipo("Thor", "IronMan", "Thanos")
competencia.agregar_equipo("Polea", "Enzo", "Nebil")
competencia.agregar_equipo("Mario", "Luigi", "Bowser")
competencia.realizar_competencia()
competencia.resultados()

## Notas
- En el thread mozo, no importa si se hace primero *self.evento_cocinero.wait()* o *self.evento_genio.wait()*. wait() solo revisa que el flag interno este activado, o sea, si es que ya se llamo a set() del evento.